In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("input/"))

from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB,GaussianNB

from keras.models import Sequential
from keras.layers import Dense, Dropout,Conv2D, Flatten
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils

from sklearn.metrics import log_loss
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier,AdaBoostClassifier

import lightgbm
# Any results you write to the current directory are saved as output.

['test.csv', 'sample_submission.csv', 'santander-customer-transaction-prediction.zip', 'train.csv']


/home1/aakash.goel/.local/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
train = pd.read_csv('input/train.csv')
test = pd.read_csv('input/test.csv')
sample_sub = pd.read_csv('input/sample_submission.csv')

In [3]:
sample_sub.head()

,ID_code,target
0,test_0,0
1,test_1,0
2,test_2,0
3,test_3,0
4,test_4,0


In [4]:
print(train.head())
print("==")
print(train.shape)
print("===")
print(test.head())
print("==")
print(test.shape)

   ID_code  target    var_0   var_1    var_2   var_3    var_4   var_5   var_6  \
0  train_0       0   8.9255 -6.7863  11.9081  5.0930  11.4607 -9.2834  5.1187   
1  train_1       0  11.5006 -4.1473  13.8588  5.3890  12.3622  7.0433  5.6208   
2  train_2       0   8.6093 -2.7457  12.0805  7.8928  10.5825 -9.0837  6.9427   
3  train_3       0  11.0604 -2.1518   8.9522  7.1957  12.5846 -1.8361  5.8428   
4  train_4       0   9.8369 -1.4834  12.8746  6.6375  12.2772  2.4486  5.9405   

     var_7   ...     var_190  var_191  var_192  var_193  var_194  var_195  \
0  18.6266   ...      4.4354   3.9642   3.1364   1.6910  18.5227  -2.3978   
1  16.5338   ...      7.6421   7.7214   2.5837  10.9516  15.4305   2.0339   
2  14.6155   ...      2.9057   9.7905   1.6704   1.6858  21.6042   3.1417   
3  14.9250   ...      4.4666   4.7433   0.7178   1.4214  23.0347  -1.2706   
4  19.2514   ...     -1.4905   9.5214  -0.1508   9.1942  13.2876  -1.5121   

   var_196  var_197  var_198  var_199  
0   7.8784

In [5]:
train['target'].value_counts(normalize=True)

0    0.89951
1    0.10049
Name: target, dtype: float64

In [6]:
independent_feat = ['var_'+str(i) for i in range(200)]
dependent_feat = 'target'

In [7]:
train[independent_feat].head()

,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,-4.9200,5.7470,...,4.4354,3.9642,3.1364,1.6910,18.5227,-2.3978,7.8784,8.5635,12.7803,-1.0914
1,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,3.1468,8.0851,...,7.6421,7.7214,2.5837,10.9516,15.4305,2.0339,8.1267,8.7889,18.3560,1.9518
2,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,-4.9193,5.9525,...,2.9057,9.7905,1.6704,1.6858,21.6042,3.1417,-6.5213,8.2675,14.7222,0.3965
3,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,-5.8609,8.2450,...,4.4666,4.7433,0.7178,1.4214,23.0347,-1.2706,-2.9275,10.2922,17.9697,-8.9996
4,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,6.2654,7.6784,...,-1.4905,9.5214,-0.1508,9.1942,13.2876,-1.5121,3.9267,9.5031,17.9974,-8.8104


In [8]:
# sc = StandardScaler()
# train[independent_feat] = sc.fit_transform(train[independent_feat])
# test[independent_feat] = sc.transform(test[independent_feat])

In [9]:
train_1, val = train_test_split(train, test_size=0.20, stratify=train[dependent_feat])
print(train.shape)
print(val.shape)

(200000, 202)
(40000, 202)


In [10]:
## Data Augmentation
# https://www.kaggle.com/jiweiliu/lgb-2-leaves-augment

def augment(x,y,t=2):
    xs,xn = [],[]
    for i in range(t):
        mask = y>0
        x1 = x[mask].copy()
        ids = np.arange(x1.shape[0])
        for c in range(x1.shape[1]):
            np.random.shuffle(ids)
            x1[:,c] = x1[ids][:,c]
        xs.append(x1)

    for i in range(t//2):
        mask = y==0
        x1 = x[mask].copy()
        ids = np.arange(x1.shape[0])
        for c in range(x1.shape[1]):
            np.random.shuffle(ids)
            x1[:,c] = x1[ids][:,c]
        xn.append(x1)

    xs = np.vstack(xs)
    xn = np.vstack(xn)
    ys = np.ones(xs.shape[0])
    yn = np.zeros(xn.shape[0])
    x = np.vstack([x,xs,xn])
    y = np.concatenate([y,ys,yn])
    return x,y

In [11]:
train_1_aug_x,train_1_aug_y = augment(train_1[independent_feat].values,train_1[dependent_feat].values)

In [12]:
train_data = lightgbm.Dataset(train_1_aug_x, label=train_1_aug_y)
val_data = lightgbm.Dataset(val[independent_feat], label=val[dependent_feat])

In [13]:
# https://www.kaggle.com/deepak525/best-parameters-lb-0-900

# V-01
# param = {'num_leaves': 13,
#          'min_data_in_leaf': 80,
#           'min_sum_hessian_in_leaf': 10.0,
#          'objective': 'binary',
#           'boost_from_average': False,
#          'max_depth': -1,
#          'learning_rate': 0.0083,
#          'boost': 'gbdt',
#          'bagging_freq': 5,
#          'tree_learner': "serial",
#          'bagging_fraction': 0.333,
#          'feature_fraction': 0.041,
#          #'reg_alpha': 1.738,
#          #'reg_lambda': 4.99,
#          'metric': 'auc',
#          #'min_gain_to_split': 0.01077313523861969,
#          #'min_child_weight': 19.428902804238373,
#          'num_threads': 8}

# V-02

random_state = 42
param = {
    "objective" : "binary",
    "metric" : "auc",
    "boosting": 'gbdt',
    "max_depth" : -1,
    "num_leaves" : 13,
    "learning_rate" : 0.01,
    "bagging_freq": 5,
    "bagging_fraction" : 0.4,
    "feature_fraction" : 0.05,
    "min_data_in_leaf": 80,
    "min_sum_heassian_in_leaf": 10,
    "tree_learner": "serial",
    "boost_from_average": "false",
    #"lambda_l1" : 5,
    #"lambda_l2" : 5,
    "bagging_seed" : random_state,
    "verbosity" : 1,
    "seed": random_state
}

In [14]:
model = lightgbm.train(param,
                       train_data,
                       valid_sets=val_data,
                       num_boost_round=100000,
                       verbose_eval=5000, 
                       early_stopping_rounds = 10000
                       )

Training until validation scores don't improve for 10000 rounds.
[5000]	valid_0's auc: 0.899994
[10000]	valid_0's auc: 0.901965
[15000]	valid_0's auc: 0.901604
[20000]	valid_0's auc: 0.901019
Early stopping, best iteration is:
[10566]	valid_0's auc: 0.901975


In [16]:
train_pred = model.predict(train_1[independent_feat])
val_pred = model.predict(val[independent_feat])

In [17]:
print(train_pred)
print("=======")
print(val_pred)

[0.16273792 0.09871602 0.20166475 ... 0.06258245 0.00276836 0.01491397]
[0.00255034 0.01273693 0.01696486 ... 0.04357571 0.02379936 0.00229526]


In [18]:
from sklearn.metrics import roc_auc_score
print("Train auc {}".format(roc_auc_score(train_1[dependent_feat],train_pred)))
print("Val auc {}".format(roc_auc_score(val[dependent_feat],val_pred)))

Train auc 0.9301116579887014
Val auc 0.9019754064585355


In [21]:
test_pred = model.predict(test[independent_feat])

In [26]:
result = pd.DataFrame({'ID_code':test['ID_code'],'target':list(test_pred)})
result.head()

,ID_code,target
0,test_0,0.121456
1,test_1,0.276285
2,test_2,0.235812
3,test_3,0.275875
4,test_4,0.069758


In [27]:
result.to_csv('output/lgbm_augonly.csv',index=False)

References

1) https://www.kaggle.com/ezietsman/simple-python-lightgbm-example
2) https://www.kaggle.com/jiweiliu/lgb-2-leaves-augment
3) https://www.kaggle.com/deepak525/best-parameters-lb-0-900